# Import Packages and Files

In [1]:
import torch.nn as nn
import dataloader as dl
import evaluater as ev
import filemanager as fm
import NonLinearClassifier as NLC
import RotNet as RN
import trainer as tr

# Load Dataset

In [2]:
trainset, testset, classes = dl.load_cifar("./datasets")

Files already downloaded and verified
Files already downloaded and verified


In [3]:
trainloader, validloader, testloader = dl.make_dataloaders(trainset, testset, 0, 128)

# Train RotNet for Rotation Task

In [4]:
# initialize network
net = RN.RotNet(num_classes=4, num_conv_block=4, add_avg_pool=False)

rot_classes = ['original', '90 rotation', '180 rotation', '270 rotation']
criterion = nn.CrossEntropyLoss()

In [5]:
# train network with own implemented metric
rot_loss_log, rot_accuracy_log, rot_max_accuracy, rot_best_epoch = tr.adaptive_learning([0.1, 0.02, 0.004, 0.0008], 
    [60, 120, 160, 200], 0.9, 5e-4, net, trainloader, validloader, criterion, rot=['90', '180', '270']) 

[1, 60] loss: 1.124
[1, 120] loss: 0.945
[1, 180] loss: 0.867
[1, 240] loss: 0.837
[1, 300] loss: 0.800
Epoch: 1 -> Loss: 0.672101140022
Epoch: 1 -> Validation Accuracy: 67.928125
[2, 60] loss: 0.739
[2, 120] loss: 0.727
[2, 180] loss: 0.715
[2, 240] loss: 0.692
[2, 300] loss: 0.666
Epoch: 2 -> Loss: 0.598019123077
Epoch: 2 -> Validation Accuracy: 75.4125
[3, 60] loss: 0.616
[3, 120] loss: 0.624
[3, 180] loss: 0.612
[3, 240] loss: 0.617
[3, 300] loss: 0.597
Epoch: 3 -> Loss: 0.526926159859
Epoch: 3 -> Validation Accuracy: 75.396875
[4, 60] loss: 0.557
[4, 120] loss: 0.549
[4, 180] loss: 0.548
[4, 240] loss: 0.546
[4, 300] loss: 0.518
Epoch: 4 -> Loss: 0.362987548113
Epoch: 4 -> Validation Accuracy: 79.878125
[5, 60] loss: 0.508
[5, 120] loss: 0.501
[5, 180] loss: 0.507
[5, 240] loss: 0.499
[5, 300] loss: 0.477
Epoch: 5 -> Loss: 0.25920560956
Epoch: 5 -> Validation Accuracy: 79.78125
[6, 60] loss: 0.454
[6, 120] loss: 0.447
[6, 180] loss: 0.464
[6, 240] loss: 0.457
[6, 300] loss: 0.485


[45, 300] loss: 0.271
Epoch: 45 -> Loss: 0.382749795914
Epoch: 45 -> Validation Accuracy: 86.015625
[46, 60] loss: 0.236
[46, 120] loss: 0.231
[46, 180] loss: 0.235
[46, 240] loss: 0.250
[46, 300] loss: 0.247
Epoch: 46 -> Loss: 0.573059856892
Epoch: 46 -> Validation Accuracy: 85.46875
[47, 60] loss: 0.294
[47, 120] loss: 0.238
[47, 180] loss: 0.245
[47, 240] loss: 0.259
[47, 300] loss: 0.237
Epoch: 47 -> Loss: 0.295177668333
Epoch: 47 -> Validation Accuracy: 85.1125
[48, 60] loss: 0.252
[48, 120] loss: 0.222
[48, 180] loss: 0.245
[48, 240] loss: 0.247
[48, 300] loss: 0.261
Epoch: 48 -> Loss: 0.342954576015
Epoch: 48 -> Validation Accuracy: 86.60625
[49, 60] loss: 0.242
[49, 120] loss: 0.240
[49, 180] loss: 0.244
[49, 240] loss: 0.242
[49, 300] loss: 0.262
Epoch: 49 -> Loss: 0.134628370404
Epoch: 49 -> Validation Accuracy: 86.41875
[50, 60] loss: 0.222
[50, 120] loss: 0.216
[50, 180] loss: 0.258
[50, 240] loss: 0.239
[50, 300] loss: 0.263
Epoch: 50 -> Loss: 0.476134449244
Epoch: 50 -> V

[89, 240] loss: 0.065
[89, 300] loss: 0.081
Epoch: 89 -> Loss: 0.17383146286
Epoch: 89 -> Validation Accuracy: 89.20625
[90, 60] loss: 0.087
[90, 120] loss: 0.060
[90, 180] loss: 0.062
[90, 240] loss: 0.074
[90, 300] loss: 0.086
Epoch: 90 -> Loss: 0.0184506885707
Epoch: 90 -> Validation Accuracy: 88.90625
[91, 60] loss: 0.054
[91, 120] loss: 0.053
[91, 180] loss: 0.058
[91, 240] loss: 0.057
[91, 300] loss: 0.071
Epoch: 91 -> Loss: 0.0515501983464
Epoch: 91 -> Validation Accuracy: 88.584375
[92, 60] loss: 0.062
[92, 120] loss: 0.051
[92, 180] loss: 0.049
[92, 240] loss: 0.048
[92, 300] loss: 0.063
Epoch: 92 -> Loss: 0.0899088978767
Epoch: 92 -> Validation Accuracy: 88.7375
[93, 60] loss: 0.084
[93, 120] loss: 0.064
[93, 180] loss: 0.057
[93, 240] loss: 0.056
[93, 300] loss: 0.059
Epoch: 93 -> Loss: 0.00941108167171
Epoch: 93 -> Validation Accuracy: 89.175
[94, 60] loss: 0.043
[94, 120] loss: 0.044
[94, 180] loss: 0.044
[94, 240] loss: 0.051
[94, 300] loss: 0.060
Epoch: 94 -> Loss: 0.336

[132, 60] loss: 0.001
[132, 120] loss: 0.001
[132, 180] loss: 0.001
[132, 240] loss: 0.001
[132, 300] loss: 0.001
Epoch: 132 -> Loss: 0.000593781471252
Epoch: 132 -> Validation Accuracy: 90.959375
[133, 60] loss: 0.001
[133, 120] loss: 0.001
[133, 180] loss: 0.001
[133, 240] loss: 0.001
[133, 300] loss: 0.001
Epoch: 133 -> Loss: 0.000506825745106
Epoch: 133 -> Validation Accuracy: 90.95625
[134, 60] loss: 0.001
[134, 120] loss: 0.001
[134, 180] loss: 0.001
[134, 240] loss: 0.001
[134, 300] loss: 0.001
Epoch: 134 -> Loss: 0.00373760610819
Epoch: 134 -> Validation Accuracy: 91.009375
[135, 60] loss: 0.001
[135, 120] loss: 0.001
[135, 180] loss: 0.001
[135, 240] loss: 0.001
[135, 300] loss: 0.001
Epoch: 135 -> Loss: 0.00332108139992
Epoch: 135 -> Validation Accuracy: 91.053125
[136, 60] loss: 0.001
[136, 120] loss: 0.001
[136, 180] loss: 0.001
[136, 240] loss: 0.001
[136, 300] loss: 0.001
Epoch: 136 -> Loss: 0.00109633430839
Epoch: 136 -> Validation Accuracy: 90.996875
[137, 60] loss: 0.0

[174, 60] loss: 0.001
[174, 120] loss: 0.001
[174, 180] loss: 0.001
[174, 240] loss: 0.001
[174, 300] loss: 0.001
Epoch: 174 -> Loss: 0.00227957591414
Epoch: 174 -> Validation Accuracy: 91.11875
[175, 60] loss: 0.001
[175, 120] loss: 0.001
[175, 180] loss: 0.001
[175, 240] loss: 0.001
[175, 300] loss: 0.001
Epoch: 175 -> Loss: 0.00251975655556
Epoch: 175 -> Validation Accuracy: 91.096875
[176, 60] loss: 0.001
[176, 120] loss: 0.001
[176, 180] loss: 0.001
[176, 240] loss: 0.001
[176, 300] loss: 0.001
Epoch: 176 -> Loss: 0.00403359532356
Epoch: 176 -> Validation Accuracy: 91.0
[177, 60] loss: 0.001
[177, 120] loss: 0.001
[177, 180] loss: 0.001
[177, 240] loss: 0.001
[177, 300] loss: 0.001
Epoch: 177 -> Loss: 0.00445945933461
Epoch: 177 -> Validation Accuracy: 90.990625
[178, 60] loss: 0.001
[178, 120] loss: 0.001
[178, 180] loss: 0.001
[178, 240] loss: 0.001
[178, 300] loss: 0.001
Epoch: 178 -> Loss: 0.00182449817657
Epoch: 178 -> Validation Accuracy: 91.0375
[179, 60] loss: 0.001
[179, 

In [5]:
# initialize network
net_paper = RN.RotNet(num_classes=4, num_conv_block=4, add_avg_pool=False)

In [6]:
# train network with metric from paper
rot_paper_loss_log, rot_paper_accuracy_log, rot_paper_max_accuracy, rot_paper_best_epoch = tr.adaptive_learning(
    [0.1, 0.02, 0.004, 0.0008], [60, 120, 160, 200], 0.9, 5e-4, net_paper, trainloader, validloader, criterion, 
    rot=['90', '180', '270'], use_paper_metric=True)

[1, 60] loss: 1.094
[1, 120] loss: 0.903
[1, 180] loss: 0.852
[1, 240] loss: 0.802
[1, 300] loss: 0.758
Epoch: 1 -> Loss: 0.606433451176
Epoch: 1 -> Validation Accuracy: 70.3962053571
[2, 60] loss: 0.734
[2, 120] loss: 0.682
[2, 180] loss: 0.668
[2, 240] loss: 0.646
[2, 300] loss: 0.635
Epoch: 2 -> Loss: 0.75306558609
Epoch: 2 -> Validation Accuracy: 74.21875
[3, 60] loss: 0.616
[3, 120] loss: 0.571
[3, 180] loss: 0.584
[3, 240] loss: 0.587
[3, 300] loss: 0.571
Epoch: 3 -> Loss: 0.737354278564
Epoch: 3 -> Validation Accuracy: 77.5886656746
[4, 60] loss: 0.535
[4, 120] loss: 0.521
[4, 180] loss: 0.520
[4, 240] loss: 0.523
[4, 300] loss: 0.501
Epoch: 4 -> Loss: 0.710784375668
Epoch: 4 -> Validation Accuracy: 79.5324900794
[5, 60] loss: 0.488
[5, 120] loss: 0.487
[5, 180] loss: 0.480
[5, 240] loss: 0.471
[5, 300] loss: 0.482
Epoch: 5 -> Loss: 0.766321063042
Epoch: 5 -> Validation Accuracy: 81.1011904762
[6, 60] loss: 0.469
[6, 120] loss: 0.445
[6, 180] loss: 0.452
[6, 240] loss: 0.445
[6,

[44, 240] loss: 0.260
[44, 300] loss: 0.253
Epoch: 44 -> Loss: 0.194753021002
Epoch: 44 -> Validation Accuracy: 86.5792410714
[45, 60] loss: 0.231
[45, 120] loss: 0.237
[45, 180] loss: 0.248
[45, 240] loss: 0.258
[45, 300] loss: 0.255
Epoch: 45 -> Loss: 0.376779615879
Epoch: 45 -> Validation Accuracy: 87.0907738095
[46, 60] loss: 0.249
[46, 120] loss: 0.225
[46, 180] loss: 0.251
[46, 240] loss: 0.234
[46, 300] loss: 0.247
Epoch: 46 -> Loss: 0.261681675911
Epoch: 46 -> Validation Accuracy: 85.6956845238
[47, 60] loss: 0.256
[47, 120] loss: 0.238
[47, 180] loss: 0.237
[47, 240] loss: 0.241
[47, 300] loss: 0.249
Epoch: 47 -> Loss: 0.351978510618
Epoch: 47 -> Validation Accuracy: 85.6646825397
[48, 60] loss: 0.263
[48, 120] loss: 0.236
[48, 180] loss: 0.234
[48, 240] loss: 0.255
[48, 300] loss: 0.242
Epoch: 48 -> Loss: 0.425981760025
Epoch: 48 -> Validation Accuracy: 85.884796627
[49, 60] loss: 0.251
[49, 120] loss: 0.226
[49, 180] loss: 0.251
[49, 240] loss: 0.249
[49, 300] loss: 0.242
Ep

[87, 120] loss: 0.057
[87, 180] loss: 0.063
[87, 240] loss: 0.067
[87, 300] loss: 0.067
Epoch: 87 -> Loss: 0.0238839685917
Epoch: 87 -> Validation Accuracy: 88.4796626984
[88, 60] loss: 0.058
[88, 120] loss: 0.054
[88, 180] loss: 0.053
[88, 240] loss: 0.060
[88, 300] loss: 0.070
Epoch: 88 -> Loss: 0.114050105214
Epoch: 88 -> Validation Accuracy: 88.7865823413
[89, 60] loss: 0.068
[89, 120] loss: 0.050
[89, 180] loss: 0.053
[89, 240] loss: 0.056
[89, 300] loss: 0.056
Epoch: 89 -> Loss: 0.0343744195998
Epoch: 89 -> Validation Accuracy: 89.4562251984
[90, 60] loss: 0.048
[90, 120] loss: 0.045
[90, 180] loss: 0.045
[90, 240] loss: 0.056
[90, 300] loss: 0.062
Epoch: 90 -> Loss: 0.0682298839092
Epoch: 90 -> Validation Accuracy: 88.972594246
[91, 60] loss: 0.058
[91, 120] loss: 0.051
[91, 180] loss: 0.051
[91, 240] loss: 0.059
[91, 300] loss: 0.060
Epoch: 91 -> Loss: 0.0204798132181
Epoch: 91 -> Validation Accuracy: 89.2113095238
[92, 60] loss: 0.045
[92, 120] loss: 0.043
[92, 180] loss: 0.04

[129, 60] loss: 0.001
[129, 120] loss: 0.001
[129, 180] loss: 0.001
[129, 240] loss: 0.001
[129, 300] loss: 0.001
Epoch: 129 -> Loss: 0.000791005790234
Epoch: 129 -> Validation Accuracy: 91.0993303571
[130, 60] loss: 0.001
[130, 120] loss: 0.001
[130, 180] loss: 0.001
[130, 240] loss: 0.001
[130, 300] loss: 0.001
Epoch: 130 -> Loss: 0.000330537557602
Epoch: 130 -> Validation Accuracy: 91.1303323413
[131, 60] loss: 0.001
[131, 120] loss: 0.001
[131, 180] loss: 0.001
[131, 240] loss: 0.001
[131, 300] loss: 0.001
Epoch: 131 -> Loss: 0.00300463661551
Epoch: 131 -> Validation Accuracy: 91.0342261905
[132, 60] loss: 0.001
[132, 120] loss: 0.001
[132, 180] loss: 0.001
[132, 240] loss: 0.001
[132, 300] loss: 0.001
Epoch: 132 -> Loss: 0.00121771544218
Epoch: 132 -> Validation Accuracy: 91.1458333333
[133, 60] loss: 0.001
[133, 120] loss: 0.001
[133, 180] loss: 0.001
[133, 240] loss: 0.001
[133, 300] loss: 0.001
Epoch: 133 -> Loss: 0.00172150507569
Epoch: 133 -> Validation Accuracy: 91.220238095

[170, 60] loss: 0.001
[170, 120] loss: 0.001
[170, 180] loss: 0.001
[170, 240] loss: 0.001
[170, 300] loss: 0.001
Epoch: 170 -> Loss: 0.00169507414103
Epoch: 170 -> Validation Accuracy: 91.1985367063
[171, 60] loss: 0.001
[171, 120] loss: 0.001
[171, 180] loss: 0.001
[171, 240] loss: 0.001
[171, 300] loss: 0.001
Epoch: 171 -> Loss: 0.00170536339283
Epoch: 171 -> Validation Accuracy: 91.2109375
[172, 60] loss: 0.001
[172, 120] loss: 0.001
[172, 180] loss: 0.001
[172, 240] loss: 0.001
[172, 300] loss: 0.001
Epoch: 172 -> Loss: 0.00244149565697
Epoch: 172 -> Validation Accuracy: 91.121031746
[173, 60] loss: 0.001
[173, 120] loss: 0.001
[173, 180] loss: 0.001
[173, 240] loss: 0.001
[173, 300] loss: 0.001
Epoch: 173 -> Loss: 0.00119297206402
Epoch: 173 -> Validation Accuracy: 91.2357390873
[174, 60] loss: 0.001
[174, 120] loss: 0.001
[174, 180] loss: 0.001
[174, 240] loss: 0.001
[174, 300] loss: 0.001
Epoch: 174 -> Loss: 0.00208307057619
Epoch: 174 -> Validation Accuracy: 91.2512400794
[175

In [ ]:
# test RotNet
net = fm.load_net('RotNet_rotation_159_best')
rot_accuracy = ev.get_accuracy(testloader, net, ['90', '180', '270'])
class_accuracy = ev.get_class_accuracy(4, testloader, net, rot_classes, ['90', '180', '270'])

In [17]:
# test paper RotNet
paper_rot_accuracy = ev.get_accuracy(testloader, net_paper, ['90', '180', '270'])
paper_class_accuracy = ev.get_class_accuracy(4, testloader, net_paper, rot_classes, ['90', '180', '270'])

Test Accuracy:  90.597 %
Test Accuracy of original :  90.650 %
Test Accuracy of 90 rotation :  90.450 %
Test Accuracy of 180 rotation :  90.630 %
Test Accuracy of 270 rotation :  90.660 %


# Save results for Rotation Task

In [7]:
rot_result = [rot_loss_log, rot_accuracy_log, rot_max_accuracy, rot_best_epoch]
paper_rot_result = [rot_paper_loss_log, rot_paper_accuracy_log, rot_paper_max_accuracy, rot_paper_best_epoch]

fm.save_variable(rot_result, 'rotation')
fm.save_variable(paper_rot_result, 'rotation_paper')

# Load results from previous trained Rotation Task

In [11]:
load = False

if load:
    rot_loss_log, rot_accuracy_log, rot_max_accuracy, rot_best_epoch = fm.load_variable('rotation')
    rot_paper_loss_log, rot_paper_accuracy_log, rot_paper_max_accuracy, rot_paper_best_epoch = fm.load_variable(
                                                                                                        'rotation_paper')

# Train classifier on Conv Block 1

In [8]:
# initialize classifier
clf_1 = NLC.NonLinearClassifier(10, 96*16*16)

In [9]:
class1_loss_log, class1_accuracy_log, class1_rot_max_accuracy, class1_rot_best_epoch = tr.adaptive_learning([0.1, 0.02, 0.004, 0.0008], 
    [20, 40, 45, 50], 0.9, 5e-4, net, trainloader, validloader, criterion, clf_1, 1) 



[1, 60] loss: 2.138
[1, 120] loss: 1.110
[1, 180] loss: 1.005
[1, 240] loss: 0.935
[1, 300] loss: 0.893
Epoch: 1 -> Loss: 0.786150455475
Epoch: 1 -> Validation Accuracy: 69.25
[2, 60] loss: 0.629
[2, 120] loss: 0.620
[2, 180] loss: 0.619
[2, 240] loss: 0.634
[2, 300] loss: 0.609
Epoch: 2 -> Loss: 0.757728755474
Epoch: 2 -> Validation Accuracy: 71.9625
[3, 60] loss: 0.359
[3, 120] loss: 0.347
[3, 180] loss: 0.372
[3, 240] loss: 0.411
[3, 300] loss: 0.426
Epoch: 3 -> Loss: 0.613182902336
Epoch: 3 -> Validation Accuracy: 70.6875
[4, 60] loss: 0.202
[4, 120] loss: 0.190
[4, 180] loss: 0.211
[4, 240] loss: 0.245
[4, 300] loss: 0.278
Epoch: 4 -> Loss: 0.422885149717
Epoch: 4 -> Validation Accuracy: 71.4125
[5, 60] loss: 0.170
[5, 120] loss: 0.155
[5, 180] loss: 0.182
[5, 240] loss: 0.200
[5, 300] loss: 0.227
Epoch: 5 -> Loss: 0.255512356758
Epoch: 5 -> Validation Accuracy: 71.1
[6, 60] loss: 0.129
[6, 120] loss: 0.123
[6, 180] loss: 0.144
[6, 240] loss: 0.162
[6, 300] loss: 0.192
Epoch: 6 ->

Epoch: 45 -> Validation Accuracy: 76.7375
[46, 60] loss: 0.003
[46, 120] loss: 0.003
[46, 180] loss: 0.003
[46, 240] loss: 0.003
[46, 300] loss: 0.003
Epoch: 46 -> Loss: 0.00568273663521
Epoch: 46 -> Validation Accuracy: 76.65
[47, 60] loss: 0.003
[47, 120] loss: 0.003
[47, 180] loss: 0.003
[47, 240] loss: 0.003
[47, 300] loss: 0.003
Epoch: 47 -> Loss: 0.0150279402733
Epoch: 47 -> Validation Accuracy: 76.475
[48, 60] loss: 0.003
[48, 120] loss: 0.003
[48, 180] loss: 0.003
[48, 240] loss: 0.003
[48, 300] loss: 0.003
Epoch: 48 -> Loss: 0.00466996431351
Epoch: 48 -> Validation Accuracy: 76.475
[49, 60] loss: 0.003
[49, 120] loss: 0.003
[49, 180] loss: 0.003
[49, 240] loss: 0.003
[49, 300] loss: 0.003
Epoch: 49 -> Loss: 0.00909674167633
Epoch: 49 -> Validation Accuracy: 76.8625
[50, 60] loss: 0.003
[50, 120] loss: 0.003
[50, 180] loss: 0.003
[50, 240] loss: 0.003
[50, 300] loss: 0.003
Epoch: 50 -> Loss: 0.0623137652874
Epoch: 50 -> Validation Accuracy: 77.0375
highest validation accuracy: 

In [21]:
class1_accuracy = ev.get_accuracy(testloader, net, classifier=clf_1, conv_block_num=1)
class1_class_accuracy = ev.get_class_accuracy(10, testloader, net, classes, classifier=clf_1, conv_block_num=1)

Test Accuracy:  75.890 %
Test Accuracy of plane :  80.500 %
Test Accuracy of car :  85.100 %
Test Accuracy of bird :  63.700 %
Test Accuracy of cat :  56.300 %
Test Accuracy of deer :  72.400 %
Test Accuracy of dog :  66.700 %
Test Accuracy of frog :  84.700 %
Test Accuracy of horse :  79.200 %
Test Accuracy of ship :  87.000 %
Test Accuracy of truck :  83.300 %


# Train classifier on Conv Block 2

In [10]:
# initialize classifier
clf_2 = NLC.NonLinearClassifier(10, 192*8*8)

In [11]:
class2_loss_log, class2_accuracy_log, class2_rot_max_accuracy, class2_rot_best_epoch = tr.adaptive_learning([0.1, 0.02, 0.004, 0.0008], 
    [20, 40, 45, 50], 0.9, 5e-4, net, trainloader, validloader, criterion, clf_2, 2) 


[1, 60] loss: 1.782
[1, 120] loss: 0.820
[1, 180] loss: 0.734
[1, 240] loss: 0.735
[1, 300] loss: 0.681
Epoch: 1 -> Loss: 1.37701964378
Epoch: 1 -> Validation Accuracy: 76.6375
[2, 60] loss: 0.450
[2, 120] loss: 0.440
[2, 180] loss: 0.458
[2, 240] loss: 0.444
[2, 300] loss: 0.443
Epoch: 2 -> Loss: 0.461004078388
Epoch: 2 -> Validation Accuracy: 78.2875
[3, 60] loss: 0.260
[3, 120] loss: 0.246
[3, 180] loss: 0.300
[3, 240] loss: 0.292
[3, 300] loss: 0.302
Epoch: 3 -> Loss: 0.733444213867
Epoch: 3 -> Validation Accuracy: 79.15
[4, 60] loss: 0.171
[4, 120] loss: 0.155
[4, 180] loss: 0.195
[4, 240] loss: 0.201
[4, 300] loss: 0.217
Epoch: 4 -> Loss: 0.136584073305
Epoch: 4 -> Validation Accuracy: 79.0375
[5, 60] loss: 0.121
[5, 120] loss: 0.109
[5, 180] loss: 0.123
[5, 240] loss: 0.157
[5, 300] loss: 0.169
Epoch: 5 -> Loss: 0.479386091232
Epoch: 5 -> Validation Accuracy: 78.8625
[6, 60] loss: 0.140
[6, 120] loss: 0.128
[6, 180] loss: 0.128
[6, 240] loss: 0.148
[6, 300] loss: 0.189
Epoch: 6 

Epoch: 45 -> Validation Accuracy: 83.4875
[46, 60] loss: 0.003
[46, 120] loss: 0.003
[46, 180] loss: 0.003
[46, 240] loss: 0.003
[46, 300] loss: 0.003
Epoch: 46 -> Loss: 0.0702613592148
Epoch: 46 -> Validation Accuracy: 83.4625
[47, 60] loss: 0.003
[47, 120] loss: 0.003
[47, 180] loss: 0.003
[47, 240] loss: 0.003
[47, 300] loss: 0.003
Epoch: 47 -> Loss: 0.0147577226162
Epoch: 47 -> Validation Accuracy: 83.475
[48, 60] loss: 0.003
[48, 120] loss: 0.003
[48, 180] loss: 0.003
[48, 240] loss: 0.003
[48, 300] loss: 0.003
Epoch: 48 -> Loss: 0.0296002328396
Epoch: 48 -> Validation Accuracy: 83.575
[49, 60] loss: 0.003
[49, 120] loss: 0.003
[49, 180] loss: 0.003
[49, 240] loss: 0.003
[49, 300] loss: 0.003
Epoch: 49 -> Loss: 0.164586335421
Epoch: 49 -> Validation Accuracy: 83.6125
[50, 60] loss: 0.003
[50, 120] loss: 0.003
[50, 180] loss: 0.003
[50, 240] loss: 0.003
[50, 300] loss: 0.003
Epoch: 50 -> Loss: 0.196919351816
Epoch: 50 -> Validation Accuracy: 83.375
highest validation accuracy: 83.7

In [20]:
class2_accuracy = ev.get_accuracy(testloader, net, classifier=clf_2, conv_block_num=2)
class2_class_accuracy = ev.get_class_accuracy(10, testloader, net, classes, classifier=clf_2, conv_block_num=2)

Test Accuracy:  82.340 %
Test Accuracy of plane :  83.200 %
Test Accuracy of car :  90.700 %
Test Accuracy of bird :  73.900 %
Test Accuracy of cat :  67.700 %
Test Accuracy of deer :  81.000 %
Test Accuracy of dog :  72.800 %
Test Accuracy of frog :  88.100 %
Test Accuracy of horse :  85.100 %
Test Accuracy of ship :  91.600 %
Test Accuracy of truck :  89.300 %


# Train classifier on Conv Block 3

In [12]:
# initialize classifier
clf_3 = NLC.NonLinearClassifier(10, 192*8*8)

In [13]:
class3_loss_log, class3_accuracy_log, class3_rot_max_accuracy, class3_rot_best_epoch = tr.adaptive_learning([0.1, 0.02, 0.004, 0.0008], 
    [20, 40, 45, 50], 0.9, 5e-4, net, trainloader, validloader, criterion, clf_3, 3) 

[1, 60] loss: 1.941
[1, 120] loss: 0.965
[1, 180] loss: 0.874
[1, 240] loss: 0.829
[1, 300] loss: 0.815
Epoch: 1 -> Loss: 0.705310165882
Epoch: 1 -> Validation Accuracy: 71.4625
[2, 60] loss: 0.606
[2, 120] loss: 0.629
[2, 180] loss: 0.575
[2, 240] loss: 0.606
[2, 300] loss: 0.622
Epoch: 2 -> Loss: 0.982685863972
Epoch: 2 -> Validation Accuracy: 73.95
[3, 60] loss: 0.400
[3, 120] loss: 0.421
[3, 180] loss: 0.443
[3, 240] loss: 0.461
[3, 300] loss: 0.477
Epoch: 3 -> Loss: 0.354474246502
Epoch: 3 -> Validation Accuracy: 73.6
[4, 60] loss: 0.282
[4, 120] loss: 0.273
[4, 180] loss: 0.312
[4, 240] loss: 0.361
[4, 300] loss: 0.382
Epoch: 4 -> Loss: 0.179344892502
Epoch: 4 -> Validation Accuracy: 74.3625
[5, 60] loss: 0.198
[5, 120] loss: 0.191
[5, 180] loss: 0.227
[5, 240] loss: 0.267
[5, 300] loss: 0.310
Epoch: 5 -> Loss: 0.821334242821
Epoch: 5 -> Validation Accuracy: 73.0625
[6, 60] loss: 0.214
[6, 120] loss: 0.179
[6, 180] loss: 0.208
[6, 240] loss: 0.253
[6, 300] loss: 0.278
Epoch: 6 ->

Epoch: 45 -> Validation Accuracy: 76.175
[46, 60] loss: 0.003
[46, 120] loss: 0.003
[46, 180] loss: 0.003
[46, 240] loss: 0.003
[46, 300] loss: 0.003
Epoch: 46 -> Loss: 0.0489293634892
Epoch: 46 -> Validation Accuracy: 76.075
[47, 60] loss: 0.003
[47, 120] loss: 0.003
[47, 180] loss: 0.003
[47, 240] loss: 0.003
[47, 300] loss: 0.003
Epoch: 47 -> Loss: 0.00955551862717
Epoch: 47 -> Validation Accuracy: 76.075
[48, 60] loss: 0.003
[48, 120] loss: 0.003
[48, 180] loss: 0.003
[48, 240] loss: 0.003
[48, 300] loss: 0.003
Epoch: 48 -> Loss: 0.3059720397
Epoch: 48 -> Validation Accuracy: 76.2625
[49, 60] loss: 0.003
[49, 120] loss: 0.003
[49, 180] loss: 0.003
[49, 240] loss: 0.003
[49, 300] loss: 0.003
Epoch: 49 -> Loss: 0.125731140375
Epoch: 49 -> Validation Accuracy: 76.3125
[50, 60] loss: 0.003
[50, 120] loss: 0.003
[50, 180] loss: 0.003
[50, 240] loss: 0.003
[50, 300] loss: 0.003
Epoch: 50 -> Loss: 0.0153604447842
Epoch: 50 -> Validation Accuracy: 76.3125
highest validation accuracy: 76.78

In [22]:
class3_accuracy = ev.get_accuracy(testloader, net, classifier=clf_3, conv_block_num=3)
class3_class_accuracy = ev.get_class_accuracy(10, testloader, net, classes, classifier=clf_3, conv_block_num=3)

Test Accuracy:  75.920 %
Test Accuracy of plane :  79.300 %
Test Accuracy of car :  85.500 %
Test Accuracy of bird :  67.500 %
Test Accuracy of cat :  56.000 %
Test Accuracy of deer :  74.400 %
Test Accuracy of dog :  67.400 %
Test Accuracy of frog :  80.900 %
Test Accuracy of horse :  80.500 %
Test Accuracy of ship :  84.400 %
Test Accuracy of truck :  83.300 %


# Train classifier on Conv Block 4

In [14]:
# initialize classifier
clf_4 = NLC.NonLinearClassifier(10, 192*8*8)

class4_loss_log, class4_accuracy_log, class4_rot_max_accuracy, class4_rot_best_epoch = tr.adaptive_learning([0.1, 0.02, 0.004, 0.0008], 
    [20, 40, 45, 50], 0.9, 5e-4, net, trainloader, validloader, criterion, clf_4, 4) 

[1, 60] loss: 2.740
[1, 120] loss: 1.802
[1, 180] loss: 1.745
[1, 240] loss: 1.702
[1, 300] loss: 1.673
Epoch: 1 -> Loss: 1.53582561016
Epoch: 1 -> Validation Accuracy: 40.625
[2, 60] loss: 1.573
[2, 120] loss: 1.576
[2, 180] loss: 1.546
[2, 240] loss: 1.571
[2, 300] loss: 1.546
Epoch: 2 -> Loss: 1.86580741405
Epoch: 2 -> Validation Accuracy: 43.225
[3, 60] loss: 1.486
[3, 120] loss: 1.479
[3, 180] loss: 1.502
[3, 240] loss: 1.508
[3, 300] loss: 1.515
Epoch: 3 -> Loss: 1.34236311913
Epoch: 3 -> Validation Accuracy: 44.4875
[4, 60] loss: 1.430
[4, 120] loss: 1.439
[4, 180] loss: 1.466
[4, 240] loss: 1.479
[4, 300] loss: 1.482
Epoch: 4 -> Loss: 1.84509682655
Epoch: 4 -> Validation Accuracy: 44.65
[5, 60] loss: 1.400
[5, 120] loss: 1.400
[5, 180] loss: 1.424
[5, 240] loss: 1.444
[5, 300] loss: 1.461
Epoch: 5 -> Loss: 2.12384462357
Epoch: 5 -> Validation Accuracy: 45.1875
[6, 60] loss: 1.394
[6, 120] loss: 1.395
[6, 180] loss: 1.390
[6, 240] loss: 1.394
[6, 300] loss: 1.437
Epoch: 6 -> Los

[46, 120] loss: 0.409
[46, 180] loss: 0.397
[46, 240] loss: 0.414
[46, 300] loss: 0.412
Epoch: 46 -> Loss: 0.2938567698
Epoch: 46 -> Validation Accuracy: 44.8875
[47, 60] loss: 0.393
[47, 120] loss: 0.375
[47, 180] loss: 0.406
[47, 240] loss: 0.384
[47, 300] loss: 0.389
Epoch: 47 -> Loss: 0.600066483021
Epoch: 47 -> Validation Accuracy: 44.55
[48, 60] loss: 0.376
[48, 120] loss: 0.389
[48, 180] loss: 0.392
[48, 240] loss: 0.379
[48, 300] loss: 0.400
Epoch: 48 -> Loss: 0.811067819595
Epoch: 48 -> Validation Accuracy: 45.2625
[49, 60] loss: 0.373
[49, 120] loss: 0.377
[49, 180] loss: 0.382
[49, 240] loss: 0.391
[49, 300] loss: 0.378
Epoch: 49 -> Loss: 1.1919503212
Epoch: 49 -> Validation Accuracy: 44.8375
[50, 60] loss: 0.370
[50, 120] loss: 0.378
[50, 180] loss: 0.379
[50, 240] loss: 0.381
[50, 300] loss: 0.392
Epoch: 50 -> Loss: 1.34228003025
Epoch: 50 -> Validation Accuracy: 45.0125
highest validation accuracy: 47.550 was achieved at epoch: 22
Finished Training


In [23]:
class4_accuracy = ev.get_accuracy(testloader, net, classifier=clf_4, conv_block_num=4)
class4_class_accuracy = ev.get_class_accuracy(10, testloader, net, classes, classifier=clf_4, conv_block_num=4)

Test Accuracy:  44.270 %
Test Accuracy of plane :  52.600 %
Test Accuracy of car :  55.000 %
Test Accuracy of bird :  34.800 %
Test Accuracy of cat :  29.300 %
Test Accuracy of deer :  38.100 %
Test Accuracy of dog :  33.500 %
Test Accuracy of frog :  48.600 %
Test Accuracy of horse :  45.400 %
Test Accuracy of ship :  55.300 %
Test Accuracy of truck :  50.100 %


# Train with Paper metric

In [15]:
# initialize classifier
paper_clf_1 = NLC.NonLinearClassifier(10, 96*16*16)
paper_clf_2 = NLC.NonLinearClassifier(10, 192*8*8)
paper_clf_3 = NLC.NonLinearClassifier(10, 192*8*8)
paper_clf_4 = NLC.NonLinearClassifier(10, 192*8*8)

In [24]:
# train
paper_class1_loss_log, paper_class1_accuracy_log, paper_class1_rot_max_accuracy, paper_class1_rot_best_epoch = tr.adaptive_learning([0.1, 0.02, 0.004, 0.0008], 
    [20, 40, 45, 50], 0.9, 5e-4, net_paper, trainloader, validloader, criterion, paper_clf_1, 1, use_paper_metric=True) 

paper_class2_loss_log, paper_class2_accuracy_log, paper_class2_rot_max_accuracy, paper_class2_rot_best_epoch = tr.adaptive_learning([0.1, 0.02, 0.004, 0.0008], 
    [20, 40, 45, 50], 0.9, 5e-4, net_paper, trainloader, validloader, criterion, paper_clf_2, 2, use_paper_metric=True) 

paper_class3_loss_log, paper_class3_accuracy_log, paper_class3_rot_max_accuracy, paper_class3_rot_best_epoch = tr.adaptive_learning([0.1, 0.02, 0.004, 0.0008], 
    [20, 40, 45, 50], 0.9, 5e-4, net_paper, trainloader, validloader, criterion, paper_clf_3, 3, use_paper_metric=True) 

paper_class4_loss_log, paper_class4_accuracy_log, paper_class4_rot_max_accuracy, paper_class4_rot_best_epoch = tr.adaptive_learning([0.1, 0.02, 0.004, 0.0008], 
    [20, 40, 45, 50], 0.9, 5e-4, net_paper, trainloader, validloader, criterion, paper_clf_4, 4, use_paper_metric=True) 

[1, 60] loss: 0.003
[1, 120] loss: 0.100
[1, 180] loss: 0.541
[1, 240] loss: 0.507
[1, 300] loss: 0.471
Epoch: 1 -> Loss: 0.668682456017
Epoch: 1 -> Validation Accuracy: 72.1850198413
[2, 60] loss: 0.282
[2, 120] loss: 0.261
[2, 180] loss: 0.295
[2, 240] loss: 0.300
[2, 300] loss: 0.301
Epoch: 2 -> Loss: 0.41001868248
Epoch: 2 -> Validation Accuracy: 73.5243055556
[3, 60] loss: 0.144
[3, 120] loss: 0.122
[3, 180] loss: 0.144
[3, 240] loss: 0.172
[3, 300] loss: 0.181
Epoch: 3 -> Loss: 1.02681064606
Epoch: 3 -> Validation Accuracy: 73.3010912698
[4, 60] loss: 0.171
[4, 120] loss: 0.139
[4, 180] loss: 0.137
[4, 240] loss: 0.143
[4, 300] loss: 0.165
Epoch: 4 -> Loss: 0.254809170961
Epoch: 4 -> Validation Accuracy: 73.4250992063
[5, 60] loss: 0.121
[5, 120] loss: 0.105
[5, 180] loss: 0.117
[5, 240] loss: 0.124
[5, 300] loss: 0.145
Epoch: 5 -> Loss: 0.536601245403
Epoch: 5 -> Validation Accuracy: 73.685515873
[6, 60] loss: 0.165
[6, 120] loss: 0.107
[6, 180] loss: 0.108
[6, 240] loss: 0.130


[44, 180] loss: 0.003
[44, 240] loss: 0.003
[44, 300] loss: 0.003
Epoch: 44 -> Loss: 0.145737975836
Epoch: 44 -> Validation Accuracy: 77.2941468254
[45, 60] loss: 0.003
[45, 120] loss: 0.003
[45, 180] loss: 0.003
[45, 240] loss: 0.003
[45, 300] loss: 0.003
Epoch: 45 -> Loss: 0.0143386721611
Epoch: 45 -> Validation Accuracy: 77.2817460317
[46, 60] loss: 0.003
[46, 120] loss: 0.003
[46, 180] loss: 0.003
[46, 240] loss: 0.003
[46, 300] loss: 0.003
Epoch: 46 -> Loss: 0.0446787774563
Epoch: 46 -> Validation Accuracy: 77.2445436508
[47, 60] loss: 0.003
[47, 120] loss: 0.003
[47, 180] loss: 0.003
[47, 240] loss: 0.003
[47, 300] loss: 0.003
Epoch: 47 -> Loss: 0.0277641117573
Epoch: 47 -> Validation Accuracy: 77.3933531746
[48, 60] loss: 0.003
[48, 120] loss: 0.003
[48, 180] loss: 0.003
[48, 240] loss: 0.003
[48, 300] loss: 0.003
Epoch: 48 -> Loss: 0.0123901367188
Epoch: 48 -> Validation Accuracy: 77.1949404762
[49, 60] loss: 0.003
[49, 120] loss: 0.003
[49, 180] loss: 0.003
[49, 240] loss: 0.0

[37, 60] loss: 0.003
[37, 120] loss: 0.003
[37, 180] loss: 0.003
[37, 240] loss: 0.003
[37, 300] loss: 0.003
Epoch: 37 -> Loss: 0.023975610733
Epoch: 37 -> Validation Accuracy: 83.2961309524
[38, 60] loss: 0.003
[38, 120] loss: 0.003
[38, 180] loss: 0.003
[38, 240] loss: 0.003
[38, 300] loss: 0.003
Epoch: 38 -> Loss: 0.0569489896297
Epoch: 38 -> Validation Accuracy: 83.345734127
[39, 60] loss: 0.004
[39, 120] loss: 0.004
[39, 180] loss: 0.004
[39, 240] loss: 0.003
[39, 300] loss: 0.003
Epoch: 39 -> Loss: 0.0194591283798
Epoch: 39 -> Validation Accuracy: 83.2837301587
[40, 60] loss: 0.003
[40, 120] loss: 0.003
[40, 180] loss: 0.003
[40, 240] loss: 0.003
[40, 300] loss: 0.003
Epoch: 40 -> Loss: 0.0164050161839
Epoch: 40 -> Validation Accuracy: 83.3581349206
[41, 60] loss: 0.003
[41, 120] loss: 0.003
[41, 180] loss: 0.003
[41, 240] loss: 0.003
[41, 300] loss: 0.003
Epoch: 41 -> Loss: 0.45476436615
Epoch: 41 -> Validation Accuracy: 83.4573412698
[42, 60] loss: 0.004
[42, 120] loss: 0.003
[

[30, 60] loss: 0.005
[30, 120] loss: 0.005
[30, 180] loss: 0.006
[30, 240] loss: 0.005
[30, 300] loss: 0.005
Epoch: 30 -> Loss: 0.0114352703094
Epoch: 30 -> Validation Accuracy: 75.5456349206
[31, 60] loss: 0.004
[31, 120] loss: 0.004
[31, 180] loss: 0.004
[31, 240] loss: 0.004
[31, 300] loss: 0.004
Epoch: 31 -> Loss: 0.0644256472588
Epoch: 31 -> Validation Accuracy: 76.0540674603
[32, 60] loss: 0.005
[32, 120] loss: 0.004
[32, 180] loss: 0.004
[32, 240] loss: 0.004
[32, 300] loss: 0.004
Epoch: 32 -> Loss: 0.139583438635
Epoch: 32 -> Validation Accuracy: 76.0044642857
[33, 60] loss: 0.008
[33, 120] loss: 0.005
[33, 180] loss: 0.005
[33, 240] loss: 0.005
[33, 300] loss: 0.005
Epoch: 33 -> Loss: 0.0959521532059
Epoch: 33 -> Validation Accuracy: 76.1408730159
[34, 60] loss: 0.008
[34, 120] loss: 0.005
[34, 180] loss: 0.005
[34, 240] loss: 0.005
[34, 300] loss: 0.005
Epoch: 34 -> Loss: 0.00555408000946
Epoch: 34 -> Validation Accuracy: 75.9672619048
[35, 60] loss: 0.004
[35, 120] loss: 0.0

[23, 60] loss: 0.776
[23, 120] loss: 0.790
[23, 180] loss: 0.787
[23, 240] loss: 0.822
[23, 300] loss: 0.836
Epoch: 23 -> Loss: 0.958708167076
Epoch: 23 -> Validation Accuracy: 43.5143849206
[24, 60] loss: 0.732
[24, 120] loss: 0.755
[24, 180] loss: 0.784
[24, 240] loss: 0.802
[24, 300] loss: 0.828
Epoch: 24 -> Loss: 1.34176528454
Epoch: 24 -> Validation Accuracy: 43.2415674603
[25, 60] loss: 0.708
[25, 120] loss: 0.733
[25, 180] loss: 0.762
[25, 240] loss: 0.772
[25, 300] loss: 0.816
Epoch: 25 -> Loss: 1.71698641777
Epoch: 25 -> Validation Accuracy: 42.7331349206
[26, 60] loss: 0.714
[26, 120] loss: 0.735
[26, 180] loss: 0.758
[26, 240] loss: 0.781
[26, 300] loss: 0.784
Epoch: 26 -> Loss: 1.58234167099
Epoch: 26 -> Validation Accuracy: 43.216765873
[27, 60] loss: 0.694
[27, 120] loss: 0.710
[27, 180] loss: 0.741
[27, 240] loss: 0.773
[27, 300] loss: 0.808
Epoch: 27 -> Loss: 1.41395783424
Epoch: 27 -> Validation Accuracy: 42.9439484127
[28, 60] loss: 0.705
[28, 120] loss: 0.708
[28, 18

In [26]:
paper_class1_accuracy = ev.get_accuracy(testloader, net_paper, classifier=paper_clf_1, conv_block_num=1, use_paper_metric=True)
paper_class1_class_accuracy = ev.get_class_accuracy(10, testloader, net_paper, classes, classifier=paper_clf_1, conv_block_num=1)

paper_class2_accuracy = ev.get_accuracy(testloader, net_paper, classifier=paper_clf_2, conv_block_num=2, use_paper_metric=True)
paper_class2_class_accuracy = ev.get_class_accuracy(10, testloader, net_paper, classes, classifier=paper_clf_2, conv_block_num=2)

paper_class3_accuracy = ev.get_accuracy(testloader, net_paper, classifier=paper_clf_3, conv_block_num=3, use_paper_metric=True)
paper_class3_class_accuracy = ev.get_class_accuracy(10, testloader, net_paper, classes, classifier=paper_clf_3, conv_block_num=3)

paper_class4_accuracy = ev.get_accuracy(testloader, net_paper, classifier=paper_clf_4, conv_block_num=4, use_paper_metric=True)
paper_class4_class_accuracy = ev.get_class_accuracy(10, testloader, net_paper, classes, classifier=paper_clf_4, conv_block_num=4)

Test Accuracy:  76.978 %
Test Accuracy of plane :  80.500 %
Test Accuracy of car :  85.600 %
Test Accuracy of bird :  64.600 %
Test Accuracy of cat :  58.400 %
Test Accuracy of deer :  75.200 %
Test Accuracy of dog :  68.400 %
Test Accuracy of frog :  83.500 %
Test Accuracy of horse :  79.200 %
Test Accuracy of ship :  86.300 %
Test Accuracy of truck :  87.600 %
Test Accuracy:  83.109 %
Test Accuracy of plane :  85.400 %
Test Accuracy of car :  91.300 %
Test Accuracy of bird :  76.300 %
Test Accuracy of cat :  68.400 %
Test Accuracy of deer :  82.400 %
Test Accuracy of dog :  73.800 %
Test Accuracy of frog :  87.900 %
Test Accuracy of horse :  87.100 %
Test Accuracy of ship :  90.000 %
Test Accuracy of truck :  90.100 %
Test Accuracy:  75.653 %
Test Accuracy of plane :  80.400 %
Test Accuracy of car :  86.100 %
Test Accuracy of bird :  67.700 %
Test Accuracy of cat :  58.100 %
Test Accuracy of deer :  73.200 %
Test Accuracy of dog :  65.700 %
Test Accuracy of frog :  82.000 %
Test Accu

In [ ]:
paper_class1_accuracy = ev.get_accuracy(testloader, net_paper, classifier=paper_clf_1, conv_block_num=1, use_paper_metric=True)
paper_class1_class_accuracy = ev.get_class_accuracy(10, testloader, net, classes, classifier=clf_1, conv_block_num=1)

paper_class2_accuracy = ev.get_accuracy(testloader, net_paper, classifier=paper_clf_2, conv_block_num=2, use_paper_metric=True)
paper_class2_class_accuracy = ev.get_class_accuracy(10, testloader, net, classes, classifier=clf_2, conv_block_num=2)

paper_class3_accuracy = ev.get_accuracy(testloader, net_paper, classifier=clf_3, conv_block_num=3, use_paper_metric=True)
paper_class3_class_accuracy = ev.get_class_accuracy(10, testloader, net, classes, classifier=clf_3, conv_block_num=3)

paper_class3_accuracy = ev.get_accuracy(testloader, net_paper, classifier=clf_3, conv_block_num=3, use_paper_metric=True)
paper_class3_class_accuracy = ev.get_class_accuracy(10, testloader, net, classes, classifier=clf_3, conv_block_num=3)

# Cheating

In [4]:
# initialize network
net = RN.RotNet(num_classes=4, num_conv_block=4, add_avg_pool=False)

rot_classes = ['original', '90 rotation', '180 rotation', '270 rotation']
criterion = nn.CrossEntropyLoss()

In [ ]:
# train network with own implemented metric
rot_loss_log, rot_accuracy_log, rot_max_accuracy, rot_best_epoch = tr.adaptive_learning([0.1, 0.02, 0.004, 0.0008], 
    [60, 120, 160, 200], 0.9, 5e-4, net, trainloader, testloader, criterion, rot=['90', '180', '270']) 

[1, 60] loss: 1.113


In [ ]:
# test RotNet
#net = fm.load_net('RotNet_rotation_159_best')
rot_accuracy = ev.get_accuracy(testloader, net, ['90', '180', '270'])
class_accuracy = ev.get_class_accuracy(4, testloader, net, rot_classes, ['90', '180', '270'])

In [ ]:
loss_log, accuracy_log, max_accuracy, best_epoch = tr.train_all_blocks(4, 10, [0.1, 0.02, 0.004, 0.0008], [2, 4, 0, 0], 0.9, 5e-4, net, trainloader, testloader, criterion) #[20, 40, 45, 50]